# Running a FCV-Res Simulation [1]
### First, Importing required packages & libraries
**WNTR** for running EPANET simulations in Python  
**NUMPY & PANDAS** for data handling and processing  
**MatPlotLib** for plotting output  
**timeit** for execution time of EPANET simulation (To compare computational efficiency)  
**re** for searching and matching text in the .inp file using regular expressions

In [ ]:
import wntr  
import numpy as np 
import pandas as pd
import matplotlib as mpl
from matplotlib import figure
import matplotlib.pyplot as plt
import timeit 
import re

### Specifying paths for simulation files and preprocessing the input
**Warning:** *Paths in this script (and the rest of this repo) are absolute unless running the network files provided within the repo*  
Input filename (with extensions) as string.  
For running the .inp files in this repository, you can use this relative path `"../../Network-Files/Network X/"` where X is the network number 

In [ ]:
 # Replace with appropriate path and filename
directory='../../Network-Files/Network 3/'
filename='Network3_4hr_FCV-Res.inp'
name_only=filename[0:-4]
print("Selected File: ",name_only)
abs_path=directory+filename

### Parse Input File for Required Information
Some key input are needed for postprocessing purposes, and thus need to be extracted from the .inp file  
Such as:  
* Supply duration for volume calculations in FRW and VRW methods
* Number & list of demand nodes (for filtering and averaging) 
* List of base demand rates for volume calculations   
 
First, we build the network model using WNTR's network module  
then we extract the needed data from the objects built by the network model
In this case, the valve is taken as a proxy for the "demand node", since the original demand node has no demand withdrawn, and the receiving node is a reservoir  
The flowrate variable in the results of the valve is the same as the demand withdrawn at the "original" demand node in a typical EPANET simulation

In [ ]:
demand_valves=[]       # For storing list of nodes that have non-zero demands
desired_demands=[]    # For storing demand rates desired by each node for desired volume calculations

# Creates a network model object using EPANET .inp file
network=wntr.network.WaterNetworkModel(abs_path)

# Iterates over the junction list in the Network object
for valve in network.valves():

    # For all nodes that have non-zero demands
    if valve[1].setting != 0:
        # Record node ID (name) and its desired demand (base_demand) in CMS
        demand_valves.append(valve[1].name)
        desired_demands.append(valve[1].setting)

# Get the supply duration in minutes (/60) as an integer
supply_duration=int(network.options.time.duration/60)

### Running an EPANET simulation through WNTR


In [ ]:
# run simulation
sim = wntr.sim.EpanetSimulator(network)
# store results of simulation
results=sim.run_sim()

### (OPTIONAL) Running a timed simulation 
for assessing computational efficiency. Requires a number of iterations to average timing over.

In [ ]:
# # Specify number of iterations to time and average over (more= more confidence in mean time per run)
# n_iterations=1000

# # Statement to be timed: read filename, create network model, run network simulation
# timed_lines='inp_file='+"'"+abs_path+"'"
# timed_lines=timed_lines+'''
# wn = wntr.network.WaterNetworkModel(inp_file) 
# wntr.sim.EpanetSimulator(wn)
# '''

# # Time and average over number of iterations
# time=np.round(timeit.timeit(stmt=timed_lines,setup='import wntr',number=n_iterations)/n_iterations*1000,decimals=2)
# print("Time taken for ",filename,' is ', time, 'milliseconds per run')

### Extracting and filtering the results
The results object produce by WNTR has a .node module which stores the values of several variables including Demand (cum) which is the flow rate withdrawn in this time step  
We extract the demands into a DataFrame indexed by time and columns represent nodes  
Nodes that were assigned demands were identified earlier, and thus any other nodes are filtered out

In [ ]:
timesrs_flowrates=pd.DataFrame()
timesrs_flowrates[0]=results.link['flowrate'].loc[0,:]
for i in range(1,supply_duration+1):
    # Extract Node Pressures from Results
    timesrs_flowrates=pd.concat([timesrs_flowrates,results.link['flowrate'].loc[i*60,:]],axis=1)

# Transpose DataFrame such that indices are time (sec) and Columns are each Node
timesrs_flowrates=timesrs_flowrates.T
# Filter DataFrame for Columns that contain Data for demand nodes only
timesrs_flowrates=timesrs_flowrates[demand_valves]

### FRW-specific Processing
For FRW methods, volumes are not a direct output of the simulation, but instead are calculated based on demands withdrawn at all time steps  
This is not required for VRW methods 

In [ ]:
# Calculates the total demand volume in the specified supply cycle
desired_volumes=[]

# Loop over each desired demand
for demand in desired_demands:
    # Append the corresponding desired volume (cum) = demand (LPS) *60 sec/min * supply duration (min) / 1000 (L/cum)
    desired_volumes.append(float(demand)*60*float(supply_duration))

# Combine demands (LPS) to their corresponding desired volume (cum)
desired_volumes=dict(zip(demand_valves,desired_volumes))

# Initalized DataFrame for storing volumes received by each demand node as a timeseries
timesrs_satisfaction=pd.DataFrame(index=timesrs_flowrates.index,columns=desired_volumes.keys())
# Set Initial volume for all consumers at 0
timesrs_satisfaction.iloc[0,:]=0

# Loop over consumers and time steps to add up volumes as a percentage of total desired volume (Satisfaction Ratio)
for timestep in list(timesrs_satisfaction.index)[1:]:
    for node in timesrs_satisfaction.columns:
        # Cummulatively add the percent satisfaction ratio (SR) increased each time step
        ## SR at time t = SR at time t-1 + demand at time t-1 (cms) *60 seconds per time step/ Desired Demand Volume (cum)
        timesrs_satisfaction.at[timestep,node]=timesrs_satisfaction.at[timestep-60,node]+timesrs_flowrates.at[timestep-60,node]*60/desired_volumes[node]

### Calculating Consumer Satisfaction Statistics
To get a better grasp of how the demands were distributed among consumers, we calculate some descriptive stats at each time step  
**Mean Satisfaction** describes the average state throughout the network, only reaches 100 if all demands are satisfied  
**Low percentile, Median, and High Percentile** at each time step for a sense of the shape of the distribution (for equality considerations) with time  
Choose the low percentile number desired (e.g., 10<sup>th</sup> percentile) and the high percentile (e.g., 90<sup>th</sup> Percentile) by changing their respective values below 

In [ ]:
# Intialize Series for storing statistics
mean=pd.Series(dtype='float64')
median=pd.Series(dtype='float64')
low_percentile=pd.Series(dtype='float64')
high_percentile=pd.Series(dtype='float64')

# Set the percentile values to be calculated
low_percent_val=10   # Range 0 to 100 ONLY
high_percent_val=90  # Range 0 to 100 ONLY

# Loop over each row (time step) in the results and calculate values of mean, median, low and high percentiles
for row in timesrs_satisfaction.index:
    mean.loc[row]=np.mean(timesrs_satisfaction.loc[row,:])*100
    low_percentile.loc[row]=np.percentile(timesrs_satisfaction.loc[row,:],low_percent_val)*100
    median.loc[row]=np.percentile(timesrs_satisfaction.loc[row,:],50)*100
    high_percentile.loc[row]=np.percentile(timesrs_satisfaction.loc[row,:],high_percent_val)*100

### (Optional) Save .csv files with  Results & Stats
Comment out any files you do not need, or skip this cell entirely if you don't want any files saved to your local device

In [ ]:
# Saves Entire Results DataFrame as Filename_TimeSeries.csv in the same path
timesrs_satisfaction.to_csv(directory+name_only+"_TimeSeries.csv")

# Saves Mean Satisfaction with time as Filename_Means.csv in the same path
mean.to_csv(directory+name_only+"_Means.csv")

# Saves Median Satisfaction with time as Filename_Medians.csv in the same path
median.to_csv(directory+name_only+"_Medians.csv")

# Saves the specified low percentile (XX) values with time as Filename_XXthPercentile.csv in the same path
low_percentile.to_csv(directory+name_only+"_"+str(low_percent_val)+"thPercentile.csv")

# Saves the specified high percentile (YY) values with time as Filename_YYthPercentile.csv in the same path
high_percentile.to_csv(directory+name_only+"_"+str(high_percent_val)+"thPercentile.csv")

### Result Visualisation
First we plot the Mean satisfaction in the network with time

In [ ]:
# Setting Universal Parameters for plotting: dots per inch (dpi), text font and size, label size
mpl.rcParams['figure.dpi'] = 450
font = {'family' : 'Times',
        'weight' : 'bold',
        'size'   : 3}
mpl.rc('font', **font)
mpl.rc('xtick', labelsize=3)
mpl.rcParams['axes.linewidth'] = 0.5

# Prepping an xaxis with hr format
supply_duration_hr=supply_duration/60
xaxis=np.arange(0,supply_duration_hr+0.00001,1/60)

fig, ax=plt.subplots()
# Change figure size (and aspect ratio) by adjusting height and width here
fig.set_figwidth(1.5)
fig.set_figheight(1)

# Formatting Plot: Setting a Title, x- and y-axis limits, major and minor ticks
ax.set_title('Average Demand Satisfaction with Time')
ax.set_xlim(0,supply_duration_hr)
ax.set_ylim(0,101)
ax.set_xticks(np.arange(0,supply_duration_hr+1,4))
ax.set_xticks(np.arange(0,supply_duration_hr+1,1),minor=True)
ax.set_yticks(np.arange(0,101,25))
ax.tick_params(width=0.5)

# Data to be plotted: Mean as a percentage (hence the multiplication by 100)
# Change color by changing the string next to c= and linewidth by value
line1,=ax.plot(xaxis,mean, c='#4575b4',linewidth=0.5)
plt.xlabel('Supply Time (hr)')
plt.ylabel('Satisfaction Ratio (%)')
# Optional: show grid or turn on and off spines (Plot box sides)
# ax.grid(visible=True,which='both')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.show

### Plotting the Distribution between Consumers (Equality)
Low Percentile, Median and High Percentile are plotted together to show the distribution between consumers with time. Area between Low and High Percentiles will be shaded

In [ ]:
# Prepping an xaxis with hr format
supply_duration_hr=supply_duration/60
xaxis=np.arange(0,supply_duration_hr+0.00001,1/60)

fig, ax=plt.subplots()
# Change figure size (and aspect ratio) by adjusting height and width here
fig.set_figwidth(1.5)
fig.set_figheight(1)

# Formatting Plot: Setting a Title, x- and y-axis limits, major and minor ticks
ax.set_title('Distribution of Demand Satisfaction')
ax.set_xlim(0,supply_duration_hr)
ax.set_ylim(0,101)
ax.set_xticks(np.arange(0,supply_duration_hr+1,4))
ax.set_xticks(np.arange(0,supply_duration_hr+1,1),minor=True)
ax.set_yticks(np.arange(0,101,25))
ax.tick_params(width=0.5)


# Data to be plotted: Mean as a percentage (hence the multiplication by 100)
# Change color by changing the string next to c= and linewidth by value
line1,=ax.plot(xaxis,median, c='#4575b4',linewidth=0.5)
plt.fill_between(xaxis, y1=low_percentile, y2=high_percentile, alpha=0.4, color='#4575b4', edgecolor=None)
plt.xlabel('Supply Time (hr)')
plt.ylabel('Satisfaction Ratio (%)')
# Optional: show grid or turn on and off spines (Plot box sides)
# ax.grid(visible=True,which='both')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.show

### References:
##### [1] N. B. Gorev and I. F. Kodzhespirova, “Noniterative Implementation of Pressure-Dependent Demands Using the Hydraulic Analysis Engine of EPANET 2,” Water Resources Management, vol. 27, no. 10, pp. 3623–3630, Aug. 2013, doi: 10.1007/s11269-013-0369-1.